In [8]:
from omegaconf import OmegaConf
from pymongo import MongoClient
import pandas as pd

In [9]:
def mongo_client():
    """Funcion to turn mongo client ON"""
    config = OmegaConf.load('/home/laertio/projects/dev/tv_scan/conf/mongo-config.yaml')
    server_ip = config.main.server_ip
    port = config.main.port
    # user = config.main.user
    # passwd = config.main.passwd
    client = MongoClient(f'mongodb://{server_ip}:{port}')
    #                     username=user,
    #                     password=passwd
    return client.TV_scan

In [ ]:
def core_processor_helper(norm, lang_detec, token, coll_name):
    """Helper to run the core text processing task defined in the processing_pipe function"""
    cursor = nlp_tank[coll_name].find()
    updates = []
    for doc in cursor:
        update = {}
        text = doc['text_raw']
        #text normalization
        if norm:
            text = normalize_text(text)
            update['text_norm_first'] = text
            #language detection
            if lang_detec:
                lang = detect_lang(text)
                update['text_lang_label'] = lang[0]
                update['text_lang_result'] = lang[1]

        #tokenization
        if token:
            if lang_detec: #only tokenize portuguese
                if "pt" in update['text_lang_label']:
                    tokens = text_tokenizer(text)
                    update['token_text_norm_first'] = tokens
                else:
                    label = update['text_lang_label']
                    logger.warning("Det. language: %s. Tokenization only for PT", label)
            else:
                ld_not_proc_before = field_checker(coll_name, 'text_lang_label')
                if ld_not_proc_before:
                    logger.error("Cannot Tokenize if no Language_detection.")
                    return print("Cannot Tokenize if no Language_detection.")
                if "pt" in doc['text_lang_label']:
                    tokens = text_tokenizer(text)
                    update['token_text_norm_first'] = tokens
                else:
                    label = update['text_lang_label']
                    logger.warning("Det. language: %s. Tokenization only for PT", label)

        updates.append(UpdateOne({'_id': doc['_id']}, {'$set': update}))

    cursor.close()
    return updates

In [10]:
df = pd.read_excel('input_sample.xlsx')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14673 entries, 0 to 14672
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   MediaDesc        14673 non-null  object
 1   SolDate          14673 non-null  int64 
 2   SolDateArch      14673 non-null  int64 
 3   SolDatePub       14673 non-null  int64 
 4   SpotCode         14673 non-null  int64 
 5   SpotDesc         14673 non-null  object
 6   SitCampId        14673 non-null  int64 
 7   SitCampDesc      14673 non-null  object
 8   VehicleDesc      14673 non-null  object
 9   BrandDesc        14673 non-null  object
 10  SubBrandDesc     3770 non-null   object
 11  SectorDesc       14673 non-null  object
 12  CategoryDesc     14523 non-null  object
 13  ClassDesc        10218 non-null  object
 14  SubClassDesc     5364 non-null   object
 15  AdvertiserDesc   14673 non-null  object
 16  MediaFile        14673 non-null  int64 
 17  MediaFileOldUrl  14673 non-null

In [17]:
df.columns

Index(['MediaDesc', 'SolDate', 'SolDateArch', 'SolDatePub', 'SpotCode',
       'SpotDesc', 'SitCampId', 'SitCampDesc', 'VehicleDesc', 'BrandDesc',
       'SubBrandDesc', 'SectorDesc', 'CategoryDesc', 'ClassDesc',
       'SubClassDesc', 'AdvertiserDesc', 'MediaFile', 'MediaFileOldUrl'],
      dtype='object')

In [12]:
db = mongo_client()

In [13]:
insert = df.head().to_dict(orient='index') #records

In [14]:
insert = list(insert.values())
db['test_document'].insert_many(insert)

In [15]:
cursor = db['test_document'].find()

In [16]:
for item in cursor:
    print(item)

{'_id': ObjectId('63ceb9e012e69af577aead4c'), 'MediaDesc': 'Televisão', 'SolDate': 20220101, 'SolDateArch': 20220224, 'SolDatePub': 20220101, 'SpotCode': 412561, 'SpotDesc': 'SABADO E DOMINGO 11/15H TEATRO POLITEAMA.RTP', 'SitCampId': 2, 'SitCampDesc': 'Nova Campanha', 'VehicleDesc': 'RTP1', 'BrandDesc': 'A PEQUENA SEREIA (MUSICAL)', 'SubBrandDesc': nan, 'SectorDesc': 'ARTIGOS E SERVICOS RECREATIVOS E CULTURAIS', 'CategoryDesc': 'ESPECTACULOS', 'ClassDesc': 'TEATROS', 'SubClassDesc': nan, 'AdvertiserDesc': 'BOCA DE CENA-PROD.ARTISTICAS LDA', 'MediaFile': 412561, 'MediaFileOldUrl': 'http://e-sol.mediamonitor.pt/SOLFILES/TELEVISAO/20220101/412561.wmv'}
{'_id': ObjectId('63ceb9e012e69af577aead4d'), 'MediaDesc': 'Televisão', 'SolDate': 20220101, 'SolDateArch': 20220224, 'SolDatePub': 20220101, 'SpotCode': 412475, 'SpotDesc': 'SABADO E DOMINGO 11/15H TEATRO POLITEAMA', 'SitCampId': 2, 'SitCampDesc': 'Nova Campanha', 'VehicleDesc': 'Panda', 'BrandDesc': 'A PEQUENA SEREIA (MUSICAL)', 'SubBran